In [3]:
# ✅ Install necessary packages in Jupyter Notebook (run this once)
%pip install fastapi uvicorn nest_asyncio sqlalchemy passlib[bcrypt] jose python-multipart


# ✅ Setup and Imports
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel, EmailStr
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
from jose import JWTError, jwt
from passlib.context import CryptContext
from datetime import datetime, timedelta
from typing import List, Optional
from fastapi.middleware.cors import CORSMiddleware

# ✅ Configurations
SECRET_KEY = "your-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

DATABASE_URL = "sqlite:///./todos.db"

# ✅ Database setup
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)
Base = declarative_base()

# ✅ Password hashing context
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# ✅ Token handling
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

# ✅ Models
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    todos = relationship("ToDo", back_populates="owner")

class ToDo(Base):
    __tablename__ = "todos"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    user_id = Column(Integer, ForeignKey("users.id"))
    owner = relationship("User", back_populates="todos")

Base.metadata.create_all(bind=engine)

# ✅ Schemas
class RegisterUser(BaseModel):
    name: str
    email: EmailStr
    password: str

class ShowUser(BaseModel):
    name: str
    email: EmailStr
    class Config:
        orm_mode = True

class Token(BaseModel):
    token: str

class TodoCreate(BaseModel):
    title: str
    description: str

class TodoShow(BaseModel):
    id: int
    title: str
    description: str
    class Config:
        orm_mode = True

# ✅ Utility functions
def get_password_hash(password):
    return pwd_context.hash(password)

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def get_current_user(token: str = Depends(oauth2_scheme), db: SessionLocal = Depends(get_db)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email: str = payload.get("sub")
        if email is None:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    user = db.query(User).filter(User.email == email).first()
    if user is None:
        raise credentials_exception
    return user

# ✅ App setup
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ✅ Routes
@app.post("/register", response_model=Token)
def register(user: RegisterUser, db: SessionLocal = Depends(get_db)):
    if db.query(User).filter(User.email == user.email).first():
        raise HTTPException(status_code=400, detail="Email already registered")
    hashed_password = get_password_hash(user.password)
    new_user = User(name=user.name, email=user.email, hashed_password=hashed_password)
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    token = create_access_token({"sub": user.email})
    return {"token": token}

@app.post("/login", response_model=Token)
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: SessionLocal = Depends(get_db)):
    user = db.query(User).filter(User.email == form_data.username).first()
    if not user or not verify_password(form_data.password, user.hashed_password):
        raise HTTPException(status_code=401, detail="Invalid credentials")
    token = create_access_token({"sub": user.email})
    return {"token": token}

@app.post("/todos", response_model=TodoShow)
def create_todo(todo: TodoCreate, db: SessionLocal = Depends(get_db), current_user: User = Depends(get_current_user)):
    new_todo = ToDo(**todo.dict(), owner=current_user)
    db.add(new_todo)
    db.commit()
    db.refresh(new_todo)
    return new_todo

@app.get("/todos", response_model=List[TodoShow])
def get_todos(page: int = 1, limit: int = 10, db: SessionLocal = Depends(get_db), current_user: User = Depends(get_current_user)):
    offset = (page - 1) * limit
    todos = db.query(ToDo).filter(ToDo.owner == current_user).offset(offset).limit(limit).all()
    return todos

@app.put("/todos/{todo_id}", response_model=TodoShow)
def update_todo(todo_id: int, todo: TodoCreate, db: SessionLocal = Depends(get_db), current_user: User = Depends(get_current_user)):
    db_todo = db.query(ToDo).filter(ToDo.id == todo_id, ToDo.user_id == current_user.id).first()
    if not db_todo:
        raise HTTPException(status_code=403, detail="Forbidden")
    db_todo.title = todo.title
    db_todo.description = todo.description
    db.commit()
    db.refresh(db_todo)
    return db_todo

@app.delete("/todos/{todo_id}", status_code=204)
def delete_todo(todo_id: int, db: SessionLocal = Depends(get_db), current_user: User = Depends(get_current_user)):
    db_todo = db.query(ToDo).filter(ToDo.id == todo_id, ToDo.user_id == current_user.id).first()
    if not db_todo:
        raise HTTPException(status_code=403, detail="Forbidden")
    db.delete(db_todo)
    db.commit()
    return


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
